In [1]:
VER = 6

import pandas as pd, numpy as np
from tqdm.notebook import tqdm
tqdm.pandas()

import os, sys, pickle, glob, gc
from collections import Counter
import cudf, itertools
print('We will use RAPIDS version',cudf.__version__)

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

# from pandarallel import pandarallel

# pandarallel.initialize(nb_workers=4, progress_bar=True, use_memory_fs=False)

import xgboost as xgb
from sklearn.model_selection import GroupKFold
import polars as pl

from pyarrow.parquet import ParquetFile
import pyarrow as pa 

We will use RAPIDS version 22.10.00a+392.g1558403753


# Training

In [2]:
type_labels = {'clicks':0, 'carts':1, 'orders':2}

In [3]:
CANDIDATE_COUNT = 100

In [4]:
train_sessions = np.load("./splitted_raw_data/val_sessions_for_train.npy", allow_pickle=True)

## For each action

In [5]:
from catboost import CatBoostRanker, Pool, MetricVisualizer

for type_str in tqdm(list(type_labels.keys())):
    
#     pf = ParquetFile(f"./candidated_features/local_{type_str}_all_data.pqt") 
    
#     whole_df = pd.read_parquet(f"./candidated_features/local_{type_str}_all_data.pqt").reset_index(drop=True)
   
    whole_df = []
    batches = sorted(glob.glob(f"./candidated_features/local_{type_str}_all_data_{CANDIDATE_COUNT}candidates_p*.pqt"))
    
    for batch in tqdm(batches):
        
        batch = pd.read_parquet(batch)#.drop(labels=["candidate_rank"], axis=1)
        batch = batch[batch.session.isin(train_sessions)].reset_index(drop=True)
        positives = batch.loc[batch['label']==1]
        negatives = batch.loc[batch['label']==0].groupby("session").sample(frac=0.2,
                                                                           random_state=1337)
        whole_df.append(positives)
        whole_df.append(negatives)
        
    whole_df = pd.concat(whole_df, axis=0, ignore_index=True)
    del positives, negatives
    
    print("sampled")

    FEATURES = whole_df.columns[2 : -1]   
    
    whole_df = whole_df.sort_values('session').reset_index(drop=True)
#     whole_df["group_count"] = whole_df.groupby('session', sort=False).cumcount('candidates') + 1
    
    skf = GroupKFold(n_splits=5)
    for fold, (train_idx, valid_idx) in enumerate(skf.split(whole_df,
                                                            whole_df['label'],
                                                            groups=whole_df['session'])):
        X_train = whole_df.loc[train_idx, FEATURES]
        y_train = whole_df.loc[train_idx, 'label']
        X_valid = whole_df.loc[valid_idx, FEATURES]
        y_valid = whole_df.loc[valid_idx, 'label']

#         train_groups = whole_df.loc[train_idx, ['session', 'group_count']].groupby('session')['group_count'].last().to_numpy()
#         val_groups = whole_df.loc[valid_idx, ['session', 'group_count']].groupby('session')['group_count'].last().to_numpy()
        
        train_groups = whole_df.loc[train_idx, "session"].values
        val_groups = whole_df.loc[valid_idx, "session"].values
#         dtrain = xgb.DMatrix(X_train, y_train, group = train_groups) 
#         dvalid = xgb.DMatrix(X_valid, y_valid, group = val_groups) 

#         xgb_parms = {
#             'objective':'rank:map',
#             'eval_metric':'map@20',
#                      'tree_method':'gpu_hist',
#                     "random_state":42, 
#                     "learning_rate":0.05,
# #                     "colsample_bytree":0.9, 
# #                     "gamma": 2,
# #                   'min_child_weight': 0.1,
#                     "max_depth":4, 
# #                     "subsample":0.8
#                     }
#         model = xgb.train(xgb_parms,
#                           dtrain=dtrain,
#                           evals=[(dtrain,'train'),(dvalid,'valid')],
#                           num_boost_round=250,
# #                           early_stopping_rounds=400,
#                           verbose_eval=100)
        train = Pool(
            data=X_train,
            label=y_train,
            group_id=train_groups
        )

        test = Pool(
            data=X_valid,
            label=y_valid,
            group_id=val_groups
        )
        
        loss = "YetiRankPairwise"
        cb_parameters = {
            'iterations': 10000,
            'loss_function': loss,
            'train_dir': loss,
            'task_type': 'GPU',
            'depth': 7,
            'learning_rate': 0.025,
#             'custom_metric': ['RecallAt:top=20', 'PrecisionAt:top=20'],
#             'eval_metric': ['MAP'],
            'early_stopping_rounds': 200,
#             'metric_period':0,
            'verbose': 100,
            'random_seed': 0,
        }
        model = CatBoostRanker(**cb_parameters)
        model.fit(train,
                  eval_set=test,
#                   plot=True
                 )
        
        print(np.mean(model.eval_metrics(test, ['MAP'])['MAP']))
    

#         model.save_model(f'./models/XGB_{CANDIDATE_COUNT}candidates_fold{fold}_{type_str}.xgb')
        model.save_model(f'./models/CB_{CANDIDATE_COUNT}candidates_fold{fold}_{type_str}.cb')
        del model, X_train, y_train, X_valid, y_valid, train, test
        
        for i in range(5):
            gc.collect()
            
    del whole_df
    for i in range(5):
        gc.collect()

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/19 [00:00<?, ?it/s]

sampled


Default metric period is 5 because PFound is/are not implemented for GPU
Metric PFound is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
Metric PFound is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	test: 0.4828714	best: 0.4828714 (0)	total: 108ms	remaining: 18m 3s
100:	test: 0.5007433	best: 0.5007485 (98)	total: 9.43s	remaining: 15m 24s
200:	test: 0.5038135	best: 0.5038154 (199)	total: 18.9s	remaining: 15m 22s
300:	test: 0.5048874	best: 0.5048874 (300)	total: 28.3s	remaining: 15m 13s
400:	test: 0.5054465	best: 0.5054465 (400)	total: 37.7s	remaining: 15m 1s
500:	test: 0.5059567	best: 0.5059567 (500)	total: 47s	remaining: 14m 51s
600:	test: 0.5063829	best: 0.5063829 (600)	total: 56.3s	remaining: 14m 40s
700:	test: 0.5067294	best: 0.5067320 (696)	total: 1m 5s	remaining: 14m 30s
800:	test: 0.5071911	best: 0.5071911 (800)	total: 1m 14s	remaining: 14m 19s
900:	test: 0.5074444	best: 0.5074444 (900)	total: 1m 24s	remaining: 14m 10s
1000:	test: 0.5076991	best: 0.5076991 (1000)	total: 1m 33s	remaining: 14m 1s
1100:	test: 0.5079141	best: 0.5079141 (1100)	total: 1m 42s	remaining: 13m 51s
1200:	test: 0.5081271	best: 0.5081271 (1200)	total: 1m 52s	remaining: 13m 42s
1300:	test: 0.5083125	be

...

0:	test: 0.0676397	best: 0.0676397 (0)	total: 89.9ms	remaining: 14m 59s
100:	test: 0.0693557	best: 0.0693558 (95)	total: 8.21s	remaining: 13m 24s
200:	test: 0.0694338	best: 0.0694340 (195)	total: 16.3s	remaining: 13m 15s
300:	test: 0.0695321	best: 0.0695329 (298)	total: 24.4s	remaining: 13m 6s
400:	test: 0.0695819	best: 0.0695837 (399)	total: 32.5s	remaining: 12m 57s
500:	test: 0.0696361	best: 0.0696363 (499)	total: 40.6s	remaining: 12m 50s
600:	test: 0.0696891	best: 0.0696891 (600)	total: 48.9s	remaining: 12m 44s
700:	test: 0.0697183	best: 0.0697196 (699)	total: 57.1s	remaining: 12m 36s
800:	test: 0.0697512	best: 0.0697535 (798)	total: 1m 5s	remaining: 12m 28s
900:	test: 0.0697690	best: 0.0697715 (883)	total: 1m 13s	remaining: 12m 21s
1000:	test: 0.0697976	best: 0.0697989 (998)	total: 1m 21s	remaining: 12m 13s
1100:	test: 0.0698124	best: 0.0698165 (1088)	total: 1m 29s	remaining: 12m 6s
1200:	test: 0.0698361	best: 0.0698373 (1190)	total: 1m 38s	remaining: 11m 58s
1300:	test: 0.0698422	

Default metric period is 5 because PFound is/are not implemented for GPU
Metric PFound is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
Metric PFound is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	test: 0.0681803	best: 0.0681803 (0)	total: 92.3ms	remaining: 15m 23s
100:	test: 0.0697274	best: 0.0697321 (94)	total: 8.21s	remaining: 13m 24s
200:	test: 0.0698715	best: 0.0698720 (199)	total: 16.4s	remaining: 13m 17s
300:	test: 0.0699083	best: 0.0699094 (299)	total: 24.5s	remaining: 13m 7s
400:	test: 0.0699325	best: 0.0699336 (397)	total: 32.6s	remaining: 12m 59s
500:	test: 0.0699810	best: 0.0699812 (499)	total: 40.7s	remaining: 12m 51s
600:	test: 0.0700279	best: 0.0700279 (600)	total: 48.9s	remaining: 12m 44s
700:	test: 0.0700523	best: 0.0700530 (690)	total: 57s	remaining: 12m 36s
800:	test: 0.0700746	best: 0.0700765 (776)	total: 1m 5s	remaining: 12m 29s
900:	test: 0.0700988	best: 0.0701001 (896)	total: 1m 13s	remaining: 12m 21s
1000:	test: 0.0701147	best: 0.0701169 (983)	total: 1m 21s	remaining: 12m 13s
1100:	test: 0.0701323	best: 0.0701335 (1099)	total: 1m 29s	remaining: 12m 5s
1200:	test: 0.0701524	best: 0.0701530 (1190)	total: 1m 37s	remaining: 11m 57s
1300:	test: 0.0701573	be

Default metric period is 5 because PFound is/are not implemented for GPU
Metric PFound is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
Metric PFound is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	test: 0.0678951	best: 0.0678951 (0)	total: 91.9ms	remaining: 15m 19s
100:	test: 0.0695001	best: 0.0695050 (95)	total: 8.18s	remaining: 13m 21s
200:	test: 0.0696378	best: 0.0696378 (200)	total: 16.3s	remaining: 13m 14s
300:	test: 0.0697137	best: 0.0697143 (297)	total: 24.4s	remaining: 13m 5s
400:	test: 0.0697406	best: 0.0697412 (392)	total: 32.5s	remaining: 12m 57s
500:	test: 0.0697831	best: 0.0697855 (492)	total: 40.6s	remaining: 12m 49s
600:	test: 0.0698256	best: 0.0698309 (587)	total: 48.8s	remaining: 12m 43s
700:	test: 0.0698480	best: 0.0698507 (675)	total: 57.1s	remaining: 12m 36s
800:	test: 0.0698743	best: 0.0698756 (792)	total: 1m 5s	remaining: 12m 29s
900:	test: 0.0698982	best: 0.0698994 (887)	total: 1m 13s	remaining: 12m 21s
1000:	test: 0.0699255	best: 0.0699255 (999)	total: 1m 21s	remaining: 12m 13s
1100:	test: 0.0699429	best: 0.0699454 (1095)	total: 1m 29s	remaining: 12m 6s
1200:	test: 0.0699643	best: 0.0699645 (1198)	total: 1m 38s	remaining: 11m 58s
1300:	test: 0.0699845	

In [6]:
feat_importances = pd.Series(model.get_feature_importance(data=train), index=whole_df[FEATURES].columns)

NameError: name 'model' is not defined

In [ ]:
feat_importances.nlargest(30).plot(kind='barh')

In [ ]:
FEATURES